In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xarray as xr
from dateutil.relativedelta import relativedelta

from my_functions import read_obsfcstana_extend_datetime

In [ ]:
# expt_name = 'DAv7_M36_ASCAT_type_13_no_catdef_fp', 'DAv7_M36_ASCAT_type_2_fp_precip', 'DAv7_M36_ASCAT_type_13_test_catdef'
expt_name = 'OLv7_M36_ASCAT_type_13_comb_fp'

start_date = datetime(2015, 4, 1)
end_date = datetime(2021, 4, 1)

start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

In [ ]:
# Read in the cross mask tile information
npzfile = np.load(f'../test_data/comb_fp/{expt_name}_{start_date_str}_{end_date_str}_obsfcstana_stats.npz')
unique_tilenum = npzfile['unique_tilenum']
num_obs_smap_cm = npzfile['num_obs_smap_cm']
num_obs_ascat_cm = npzfile['num_obs_ascat_cm']

# Find the tiles with both ASCAT and SMAP observations
tilenum_both = unique_tilenum[(num_obs_smap_cm > 0) & (num_obs_ascat_cm > 0)]

In [ ]:
# Calculate the daily statistics in observation space

# Define the path directory
path_dir = f'/discover/nobackup/amfox/Experiments/{expt_name}/{expt_name}/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg'

path_dir = '../test_data'
expt_name = 'DAv7_M36_MULTI_type_13_comb_fp'
start_date = datetime(2020, 6, 30)
end_date = datetime(2020, 6, 30)
start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')


# Define the common file name start
file_name_start = f'{expt_name}.ens_avg.ldas_ObsFcstAna.'

# Define the print flag
printflag = False

# Initialize lists to store the mean values for each variable and the dates
dates_list = []

ascat_obs_obs_count_list = []
ascat_obs_obs_mean_list = []
ascat_obs_fcst_mean_list = []
ascat_obs_ana_mean_list = []
ascat_omf_mean_list = []
ascat_oma_mean_list = []
ascat_omf_max_list = []
ascat_omf_std_list = []
ascat_oma_std_list = []

smap_obs_obs_count_list = []
smap_obs_obs_mean_list = []
smap_obs_fcst_mean_list = []
smap_obs_ana_mean_list = []
smap_omf_mean_list = []
smap_oma_mean_list = []
smap_omf_max_list = []
smap_omf_std_list = []
smap_oma_std_list = []

total_obs_both_list = []

# Define the start and end dates
# start_date = datetime.strptime('20150401', '%Y%m%d')
# end_date = datetime.strptime('20210331', '%Y%m%d')

# Loop over the dates
current_date = start_date
while current_date <= end_date:
    # Define the file name for the current date
    file_name = file_name_start + current_date.strftime('%Y%m%d')
    if file_name[-4:] == '0401':
        print('file_name = ', file_name)
    print('file_name = ', file_name)
    # Call the read_obsfcstana function for the current file
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana_extend_datetime(path_dir, file_name, printflag)

    # Convert the lists to numpy arrays
    a_obs_obs = np.array(obs_obs[(obs_species > 4) & np.isin(obs_tilenum, tilenum_both)])
    a_obs_fcst = np.array(obs_fcst[(obs_species > 4) & np.isin(obs_tilenum,tilenum_both)])
    a_obs_ana = np.array(obs_ana[(obs_species > 4) & np.isin(obs_tilenum, tilenum_both)])
    
    # Calculate the mean values for the variables
    obs_obs_count = len(a_obs_obs)
    obs_obs_mean = np.mean(a_obs_obs)
    obs_fcst_mean = np.mean(a_obs_fcst)
    obs_ana_mean = np.mean(a_obs_ana)
    omf_mean = np.mean(a_obs_obs - a_obs_fcst)
    oma_mean = np.mean(a_obs_obs - a_obs_ana)
    # Calculate the maximum absolute difference between obs_obs and obs_fcst
    if a_obs_fcst.size > 0 and a_obs_obs.size > 0 and a_obs_fcst.shape == a_obs_obs.shape:
        omf_max = np.max(abs(a_obs_obs - a_obs_fcst))
    else:
        omf_max = np.nan
        print('Current date = ', current_date)
    omf_std = np.std(a_obs_obs - a_obs_fcst)
    oma_std = np.std(a_obs_obs - a_obs_ana)

    # Append the mean values to the lists
    ascat_obs_obs_count_list.append(obs_obs_count)
    ascat_obs_obs_mean_list.append(obs_obs_mean)
    ascat_obs_fcst_mean_list.append(obs_fcst_mean)
    ascat_obs_ana_mean_list.append(obs_ana_mean)
    ascat_omf_mean_list.append(omf_mean)
    ascat_oma_mean_list.append(oma_mean)
    ascat_omf_max_list.append(omf_max)
    ascat_omf_std_list.append(omf_std)
    ascat_oma_std_list.append(oma_std)
    
    # Repeat for the SMAP obs
    # Convert the lists to numpy arrays
    s_obs_obs = np.array(obs_obs[(obs_species < 5) & np.isin(obs_tilenum, tilenum_both)])
    s_obs_fcst = np.array(obs_fcst[(obs_species < 5) & np.isin(obs_tilenum, tilenum_both)])
    s_obs_ana = np.array(obs_ana[(obs_species < 5) & np.isin(obs_tilenum, tilenum_both)])
    
    # Calculate the mean values for the variables
    obs_obs_count = len(s_obs_obs)
    obs_obs_mean = np.mean(s_obs_obs)
    obs_fcst_mean = np.mean(s_obs_fcst)
    obs_ana_mean = np.mean(s_obs_ana)
    omf_mean = np.mean(s_obs_obs - s_obs_fcst)
    oma_mean = np.mean(s_obs_obs - s_obs_ana)
    # Calculate the maximum absolute difference between obs_obs and obs_fcst
    if s_obs_fcst.size > 0 and s_obs_obs.size > 0 and s_obs_fcst.shape == s_obs_obs.shape:
        omf_max = np.max(abs(s_obs_obs - s_obs_fcst))
    else:
        omf_max = np.nan
        print('Current date = ', current_date)
    omf_std = np.std(s_obs_obs - s_obs_fcst)
    oma_std = np.std(s_obs_obs - s_obs_ana)

    # Append the mean values to the lists
    smap_obs_obs_count_list.append(obs_obs_count)
    smap_obs_obs_mean_list.append(obs_obs_mean)
    smap_obs_fcst_mean_list.append(obs_fcst_mean)
    smap_obs_ana_mean_list.append(obs_ana_mean)
    smap_omf_mean_list.append(omf_mean)
    smap_oma_mean_list.append(oma_mean)
    smap_omf_max_list.append(omf_max)
    smap_omf_std_list.append(omf_std)
    smap_oma_std_list.append(oma_std)

    # Calculate the number of joint observations
    # Get all unique hour values
    date_time_np = np.array(date_time)
    unique_hours = np.unique([d['hour'][0] for d in date_time_np])

    # Initialize the total number of tiles
    total_obs_both = 0

    # Loop over all unique hour values
    for hr in unique_hours:
        s_tiles = np.array(obs_tilenum)[(obs_species < 5) & (np.array([d['hour'][0] for d in date_time_np]) == hr)]
        a_tiles = np.array(obs_tilenum)[(obs_species > 4) & (np.array([d['hour'][0] for d in date_time_np]) == hr)]
    
        # Find the intersection of s_tiles and a_tiles
        intersect_tiles = np.intersect1d(s_tiles, a_tiles)
        num_tiles_both = len(intersect_tiles)
        
        # Add the number of tiles for this hour to the total
        total_obs_both += num_tiles_both

    total_obs_both_list.append(total_obs_both)
        
    # Append the current date to the dates list
    dates_list.append(current_date.strftime('%Y%m%d'))

    # Increment the current date by one day
    current_date += timedelta(days=1)



In [ ]:
# Get all unique hour values
date_time_np = np.array(date_time)
unique_hours = np.unique([d['hour'][0] for d in date_time_np])

# Initialize the total number of tiles
total_tiles_both = 0
intersect_tiles_list = []

# Loop over all unique hour values
for hr in unique_hours:
    s_tiles = np.array(obs_tilenum)[(obs_species < 5) & (np.array([d['hour'][0] for d in date_time_np]) == hr)]
    a_tiles = np.array(obs_tilenum)[(obs_species > 4) & (np.array([d['hour'][0] for d in date_time_np]) == hr)]

    # Find the intersection of s_tiles and a_tiles
    intersect_tiles = np.intersect1d(s_tiles, a_tiles)
    num_tiles_both = len(intersect_tiles)

    print('num_tiles_both = ', num_tiles_both)

    # Add the number of tiles for this hour to the total
    total_tiles_both += num_tiles_both
    intersect_tiles_list.append(intersect_tiles)    

print('total tiles both = ', total_tiles_both)
    
# Append the current date to the dates list
dates_list.append(current_date.strftime('%Y%m%d'))

# Increment the current date by one day
current_date += timedelta(days=1)

In [ ]:
# obs_species 1 -7 correspond with these names: SMAP_L1C_Tbh_A, SMAP_L1C_Tbh_D, SMAP_L1C_Tbv_A, SMAP_L1C_Tbv_D, ASCAT_META_SM, ASCAT_METB_SM, ASCAT_METC_SM

# Create a dictionary using these names that corresponds to the obs_species values

obs_species_dict = {1: 'SMAP_L1C_Tbh_A', 2: 'SMAP_L1C_Tbh_D', 3: 'SMAP_L1C_Tbv_A', 4: 'SMAP_L1C_Tbv_D', 5: 'ASCAT_META_SM', 6: 'ASCAT_METB_SM', 7: 'ASCAT_METC_SM'}

# Make a scatter plot using obs_lon and obs_lat and color by obs_species with a different color for each obs_species

# Create a dictionary to store the colors for each obs_species
colors_dict = {1: 'red', 2: 'blue', 3: 'green', 4: 'purple', 5: 'orange', 6: 'yellow', 7: 'black'}


# Create a dictionary to store the label for each obs_species
label_dict = {1: 'SMAP_L1C_Tbh_A', 2: 'SMAP_L1C_Tbh_D', 3: 'SMAP_L1C_Tbv_A', 4: 'SMAP_L1C_Tbv_D', 5: 'ASCAT_META_SM', 6: 'ASCAT_METB_SM', 7: 'ASCAT_METC_SM'}

# Create a dictionary to store the marker size for each obs_species
marker_size_dict = {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1}

from matplotlib.lines import Line2D

# Make the plot
plt.figure(figsize=(20, 10))    

# Create a list to store the custom legend handles
legend_elements = []

for i in range(5, 8):
    obs_species_bool = np.array(obs_species, dtype=float) == i
    plt.scatter(np.array(obs_lon)[obs_species_bool], np.array(obs_lat)[obs_species_bool], c=colors_dict[i], s=marker_size_dict[i])
    
    # Create a Line2D object for the custom legend
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label=label_dict[i], markerfacecolor=colors_dict[i], markersize=8))

# Create the custom legend
plt.legend(handles=legend_elements)
plt.show()



In [ ]:
from mapper_functions import plot_global_tight

obarray = np.empty([len(obs_tilenum), 3])
obarray.fill(np.nan)
obarray[:, 1] = obs_lon
obarray[:, 2] = obs_lat

#obarray[(obs_species < 5), 0] = 1
#obarray[(obs_species > 4), 0] = 2

# Find the indicies of the intersect_tiles_list
intersect_tiles_list_indices = []
for i in range(len(intersect_tiles_list)):
    intersect_tiles_list_indices.append(np.where(np.isin(obs_tilenum, intersect_tiles_list[i]))[0])

# Plot intersect_tiles_list_indices setting obarray[:, 0] = 3
for i in range(len(intersect_tiles_list_indices)):
    obarray[intersect_tiles_list_indices[i], 0] = 3

plot_global_tight(obarray)

obarray.fill(np.nan)
obarray[:, 1] = obs_lon
obarray[:, 2] = obs_lat
obarray[(obs_species < 5), 0] = 1
plot_global_tight(obarray)

obarray.fill(np.nan)
obarray[:, 1] = obs_lon
obarray[:, 2] = obs_lat
obarray[(obs_species > 4), 0] = 2
plot_global_tight(obarray)


In [ ]:
np.savez(f'{expt_name}_{start_date_str}_{end_date_str}_OmF_ts_cross_masked.npz',
         dates_list=dates_list,
         
         ascat_obs_obs_count_list=ascat_obs_obs_count_list,
         ascat_obs_obs_mean_list=ascat_obs_obs_mean_list,
         ascat_obs_fcst_mean_list=ascat_obs_fcst_mean_list,
         ascat_obs_ana_mean_list=ascat_obs_ana_mean_list,
         ascat_omf_mean_list=ascat_omf_mean_list,
         ascat_oma_mean_list=ascat_oma_mean_list,
         ascat_omf_max_list=ascat_omf_max_list,
         ascat_omf_std_list=ascat_omf_std_list,
         ascat_oma_std_list=ascat_oma_std_list,
        
         smap_obs_obs_count_list=smap_obs_obs_count_list,
         smap_obs_obs_mean_list=smap_obs_obs_mean_list,
         smap_obs_fcst_mean_list=smap_obs_fcst_mean_list,
         smap_obs_ana_mean_list=smap_obs_ana_mean_list,
         smap_omf_mean_list=smap_omf_mean_list,
         smap_oma_mean_list=smap_oma_mean_list,
         smap_omf_max_list=smap_omf_max_list,
         smap_omf_std_list=smap_omf_std_list,
         smap_oma_std_list=smap_oma_std_list) 

In [ ]:
# Plot the time series of the count of observations for smap and ascat
plt.figure()
plt.plot(dates_list, ascat_obs_obs_count_list, label='ASCAT')
plt.plot(dates_list, smap_obs_obs_count_list, label='SMAP')
plt.xlabel('Date')
plt.ylabel('Number of observations')
plt.title('Number of observations in cross mask')
plt.legend()
plt.show()
